## Нахождение мест посадки пассажиров по транзакционным данным об облате проезда смарт-картами "Подорожник"                            

В основе данной методики лежит метод восстановления информации об остановках, где садились пассажиры, по транзакциям оплаты проезда смарт-картами. Особенность имеющихся транзакций - отсутствие информации о месте их совершения.

Базовое предположение: транзакции оплаты совершаются сразу при входе пассажира в транспортное средство.

Созданный алгоритм базируется на идеях, описанных в статье «Extracting bus transit boarding stop information using smart card transaction data» (Chen Z., Fan W.; 2018) и адаптирован под особенности системы коммерческого транспорта Санкт-Петербурга.

In [ ]:
import numpy as np
import pandas as pd

## Обработка и анализ данных. Проверка корректности

In [ ]:
# Загружаем dataset с информацией об исследуемых коммерческих маршрутах
comm_df = pd.read_csv('resources/commercial_routes.csv')
comm_df = comm_df[['route_number', 'num1', 'stops_in_direction1', 'num2', 'stops_in_direction2', 'clusters_in_direction1', 'clusters_in_direction2', 'distances1', 'distanсes2', 'sсheduled_trip_time']]

In [ ]:
# Вспомогательная функция, вычисляющая время в пути указанного маршрута между всеми парами остановок
# Результатом ее работы становится обновление информации в dataset-е коммерческих маршрутов 'comm_df'
def get_time_between_stations(route_num):
    stations_in_dist1 = np.array(comm_df[comm_df['route_number'] == route_num]['distances1'].values[0].split(',')).astype(int)
    stations_in_dist2 = np.array(comm_df[comm_df['route_number'] == route_num]['distanсes2'].values[0].split(',')).astype(int)
    
    all_route_dist1 = np.sum(stations_in_dist1)
    all_route_dist2 = np.sum(stations_in_dist2)
    
    all_route_time = comm_df[comm_df['route_number'] == route_num]['sсheduled_trip_time'].values[0]
    
    avg_speed1 = all_route_dist1 * 1.0 / all_route_time # в м/с
    avg_speed2 = all_route_dist1 * 1.0 / all_route_time # в м/с
    
    times_array1 = str(int(round((stations_in_dist1[0] * 1.0) /  avg_speed1)))
    times_array2 = str(int(round((stations_in_dist2[0] * 1.0) /  avg_speed2)))
    
    for i in range(1, stations_in_dist1.shape[0]):
        times_array1 += ',' + str(int(round((stations_in_dist1[i] * 1.0) /  avg_speed1)))
    for i in range(1, stations_in_dist2.shape[0]):
        times_array2 += ',' + str(int(round((stations_in_dist2[i] * 1.0) /  avg_speed2)))
        
    return times_array1, times_array2

times_for_stops1_arr = []
times_for_stops2_arr = []

all_comm_routes = comm_df['route_number'].values
for route in all_comm_routes:
    t1, t2 = get_time_between_stations(route)
    times_for_stops1_arr.append(t1)
    times_for_stops2_arr.append(t2)
    

# Добавление информации о временах преодоления остановок в dataframe
times_for_stops1_column = pd.Series(np.array(times_for_stops1_arr), index=comm_df.index)
comm_df.loc[:, 'times_for_stops1'] = times_for_stops1_column

times_for_stops2_column = pd.Series(np.array(times_for_stops2_arr), index=comm_df.index)
comm_df.loc[:, 'times_for_stops2'] = times_for_stops2_column

# Сохраняем обновленную информацию о коммерческих маршрутах
comm_df.to_csv('resources/commercial_routes.csv', encoding='utf-8-sig')

In [ ]:
# Перезагружаем обновленный dataset с коммерческими маршрутами для дальнейшего использования
comm_df = pd.read_csv('resources/commercial_routes.csv')
comm_df = comm_df[['route_number', 'num1', 'stops_in_direction1', 'num2', 'stops_in_direction2', 'clusters_in_direction1', 'clusters_in_direction2', 'distances1', 'distanсes2', 'sсheduled_trip_time', 'times_for_stops1', 'times_for_stops2']]
comm_df

In [ ]:
# Загружаем обработанный dataset с транзакциями
tr_df = pd.read_csv('resources/final_transactions_with_duplicate_labels.csv')

In [ ]:
# Удаляем дубликаты из данных
# Анализ данных на дубликаты проводился в 'data_preparation.py'
tr_df = tr_df[tr_df['IS_DUPLICATE'] == 0]

In [ ]:
# Проводим анализ записей и выводим некоторую статистику по данным

# Отсортированные по убыванию количества транзакций записи о маршруте, номере бортового транспорта и днях оплаты
max_transactions = tr_df.groupby(['ROUTE_NUM', 'CARRIER_BOARD_NUM', 'TRANSACT_MONTH', 'TRANSACT_DAY']).TRANSACT_TIME.count()
max_transactions = max_transactions.sort_values(ascending=False)

### NB! Некорректные значения в данных о транзакциях!
В результате анализа данных транзакций были обнаружены некорректные записи.
На коммерческом маршруте №68 количество дневных транзакций оплаты по картам у транспортного средства с бортовым номером 0 аномально большое. Потенциальной причиной возникновения некорректных данных можно считать сбой в системе в октябре 2018 года, который привел к потере информации о бортовых номерах у ряда транспортных средств, следующих по маршруту №68 и в результате произошло объединение данных о нескольких транспортных средствах (с новым общим бортовым номером 0).

Т.к. разделить такие данные не представляется возможным, их нельзя учитывать при расчете пассажиропотока по этой методике.

In [ ]:
# Готовим датасеты для анализа выбранных маршрутов
choosen_routes = ['68', '191', '92', '7', '1', '55', '67', '184', '226']

for route in choosen_routes:
    route_df = tr_df[tr_df['ROUTE_NUM'] == route]
    route_df.to_csv('resources/' + route + '_route.csv')

## Алгоритм вычисления мест посадки пассажиров

Данный алгоритм состоит из нескольких этапов. 

1. На первом этапе происходит разделение поездок. Каждой отдельной поездке назначается свой идентификатор. Отделение различных поездок друг от друга происходит по следующему принципу (см. блок-схему). Здесь параметр 12 минут был выбран практически на основе анализа имеющихся данных как оптимальный разделитель.

 ![choose_trip.png](resources/img/choose_trip.png)
 
2. На втором этапе производится кластеризация записей. Маленький временной промежуток между транзакциями означает, что совершившие их пассажиры садились в одном месте, а значит такие записи нужно отнести к одному кластеру. Более подробно данный этап описан на следующей блок-схеме:

 ![clasterization_diagram.png](resources/img/clasterization_diagram.png)

3. Целью третьего этапа является соотнесение полученных кластеров и остановок на маршруте. В результате для каждой транзакции должна появиться информация об остановке, на которой входил совершивший ее пассажир.
 Алгоритм соотнесения кластеров и остановок выглядит следующим  образом:
 1. Первому кластеру каждой поездки ставится в соответствие первая остановка маршрута (это базовое предположение, его можно попробовать поменять, если станет известна дополнительная информация о том, как это можно вычислить более точно).
 2. Обозначим за $t_{n(n+1)}$ временную разницу между первой записью в $n+1$-ом кластере и последней записью в $n$-ом кластере. Пусть кластер $n$ был соотнесен с остановкой $i$. Обозначим за $a_{i(i+1)}$ время прохождения транспортным средством по маршруту от остановки $i$ до остановки $i+1$.
 3. Если $t_{n(n+1)}<a_{i(i+1)}$, то соотнесем кластер $n+1$ с остановкой $i+1$.
 4. Если $t_{n(n+1)}>a_{i(i+1)}$, то сравним $t_{n(n+1)}$ и $a_{i(i+2)}$. Если снова больше, то будем сравнивать $t_{n(n+1)}$ с $a_{i(i+k)}$ до тех пор, пока не будет выполнено $t_{n(n+1)}<a_{i(i+k)}$. После этого соотнесем кластер $n+1$ с остановкой $i+k$.
 
 Пример для наглядности:
 
 ![boarding_extraction.png](resources/img/boarding_extraction.png)
 
Результатом работы описанного выше алгоритма будет нахождение для каждой транзакционной записи номера и направления поездки, а также остановки, на которой садился пассажир, ее совершивший.

In [ ]:
# Здесь задаются параметры алгоритму вычисления мест посадки пассажиров

# 1. Параметр, предназначенный для грубого разделения поездок транспортного средства по маршруту
# его значение - временной промежуток в секундах между записями об оплате, 
# который позволяет гарантированно отнести эти записи к разным поездкам
coarse_trips_pivot = 720 

# 2. Параметр, позволяющий понять, принадлежат ли последовательные записи одному кластеру или нет
# его значение также в секундах
clusters_pivot = 25

In [ ]:
# Вспомогательная функция, вычисляющая разность в секундах между двумя переданными временными штампами
# Допущения: timestamp2 > timestamp1, это гарантируется при подаче на вход
def get_times_delta(timestamp1, timestamp2):
    time_arr1 = timestamp1.split(':')
    time_arr2 = timestamp2.split(':')
    return (int(time_arr2[2]) + int(time_arr2[1]) * 60 + int(time_arr2[0]) * 3600) - (int(time_arr1[2]) + int(time_arr1[1]) * 60 + int(time_arr1[0]) * 3600)

#get_times_delta("05:23:00", "05:24:00")

In [ ]:
# Вспомогательная функция. Возвращает dataframe с записями об указанном маршруте в указанную дату
# В данной функции дополнительно происходит проверка на корректность информации о номере т\с
# Корректные номера т\с являются однозначными - четырехзначными. Номера, содержащие большее количество цифр - некорректны

# Вспомогательная функция. Проверяет, есть ли в указанную дату записи о данном маршруте
# Вспомогательная функция. Позволяет по номеру маршрута узнать длительность его поездки по маршруту

def get_route_df_by_date(route_num, tr_date):
    route_df = pd.read_csv('resources/' + route_num + '_route.csv')
    route_df = route_df[route_df['TRANSACT_MONTH'] == int(tr_date.split('.')[1])]
    route_df = route_df[route_df['TRANSACT_DAY'] == int(tr_date.split('.')[0])] 
    route_df = route_df[route_df['CARRIER_BOARD_NUM'] <= 10000]
    route_df = route_df.sort_values(by=['TRANSACT_YEAR', 'TRANSACT_MONTH', 'TRANSACT_DAY', 'CARRIER_BOARD_NUM', 'TRANSACT_TIME'])
    route_df = route_df.reset_index(drop=True)
    return route_df[['CARD_NUM', 'TRANSACT_DAY', 'TRANSACT_MONTH', 'TRANSACT_YEAR', 'CARRIER_BOARD_NUM', 'TRANSACT_TIME', 
                     'ROUTE_NUM']]


def check_date(route_num, tr_date):
    route_df = get_route_df_by_date(route_num, tr_date)
    return route_df['ROUTE_NUM'].count() > 0


def get_scheduled_trip_time(route_num):
    return comm_df[comm_df['route_number'] == int(route_num)]['sсheduled_trip_time'].values[0]

In [ ]:
# Вспомогательная функция. Позволяет по номеру маршрута узнать длительность его поездки по маршруту
def get_scheduled_trip_time(route_num):
    return comm_df[comm_df['route_number'] == int(route_num)]['sсheduled_trip_time'].values[0]

#get_scheduled_trip_time(68)

In [ ]:
# Вспомогательная функция. Для переданного маршрута и дня рассчитывает пассажиропотоки в течение этого дня 
# на всех транспортных средствах, которые были на маршруте в этот день

def get_boarding_locations_by_day(route_num, tr_date):
    cur_comm_df = comm_df[comm_df['route_number'] == int(route_num)]
    route_df = get_route_df_by_date(route_num, tr_date)
    board_nums_array = route_df['CARRIER_BOARD_NUM'].unique()
    for board_num in board_nums_array:
        board_df = route_df[route_df['CARRIER_BOARD_NUM'] == board_num]
        tr_amount = board_df.shape[0]
        tr_times = board_df['TRANSACT_TIME'].values
        
        # Этап 1 - разделение поездок по времени между транзакциями
        board_trips = [1]
        board_directions = [0]
        trip_id = 1
        direction = 0
        first_note_of_the_trip = 0
        for i in range(1, tr_amount):
            sheduled_trip_duration = get_scheduled_trip_time(route_num)
            if (get_times_delta(tr_times[i - 1], tr_times[i]) >= coarse_trips_pivot) or get_times_delta(tr_times[first_note_of_the_trip], tr_times[i]) >= sheduled_trip_duration:
                trip_id += 1
                first_note_of_the_trip = i
                board_trips.append(trip_id)  
                if get_times_delta(tr_times[i - 1], tr_times[i]) >= sheduled_trip_duration:
                    dir_coeff = get_times_delta(tr_times[i - 1], tr_times[i]) // sheduled_trip_duration
                    if dir_coeff % 2 == 0:
                        direction = (direction + 1) % 2
                        board_directions.append(direction)
                    else:
                        board_directions.append(direction)
                else:
                    direction = (direction + 1) % 2
                    board_directions.append(direction)
            else:
                board_trips.append(trip_id)
                board_directions.append(direction)
                
        preliminary_trip_id_column = np.array(board_trips)
        preliminary_direction_column = np.array(board_directions)
        
        # Этап 2 - кластеризация записей
        tr_times = board_df['TRANSACT_TIME'].values
        board_clusters = [1]
        cluster_id = 1
        cur_trip_num = preliminary_trip_id_column[0]
        for i in range(1, tr_amount):
            trip_id = preliminary_trip_id_column[i]
            if trip_id == cur_trip_num:
                if get_times_delta(tr_times[i - 1], tr_times[i]) < clusters_pivot:
                    board_clusters.append(cluster_id)
                else:
                    cluster_id += 1
                    board_clusters.append(cluster_id)
            else:
                cur_trip_num = trip_id
                cluster_id = 1
                board_clusters.append(cluster_id)
        
        clusters_column = np.array(board_clusters)
        
        # Этап 3 - соотнесение кластеров и информации об остановках
        tr_times = board_df['TRANSACT_TIME'].values
        boarding_stations_nums = [0]
        station_num = 0
        cur_trip_num = preliminary_trip_id_column[0]
        cur_direction = preliminary_direction_column[0]
        cur_cluster = clusters_column[0]
        
        if cur_direction == 0:
            stops_times = cur_comm_df['times_for_stops1'].values[0].split(',')
            route_stops_arr = cur_comm_df['stops_in_direction1'].values[0].split(',')
        else:
            stops_times = cur_comm_df['times_for_stops2'].values[0].split(',')
            route_stops_arr = cur_comm_df['stops_in_direction2'].values[0].split(',')
        
        boarding_stations_id = [route_stops_arr[0]]
        station_id = route_stops_arr[0]
              
        for i in range(1, tr_amount):
            trip_id = preliminary_trip_id_column[i]
            direction_id = preliminary_direction_column[i]
            cluster_id = clusters_column[i]
            if trip_id == cur_trip_num:
                if cluster_id != cur_cluster:
                    cur_cluster = cluster_id
                    delta = get_times_delta(tr_times[i - 1], tr_times[i])
                    time_between_stations = 0
                    finded = 0
                    for j in range(station_num, len(stops_times)):
                        time_between_stations += int(stops_times[j])
                        if delta < time_between_stations:
                            finded = 1
                            station_num = j + 1
                            station_id = route_stops_arr[j + 1]
                            boarding_stations_nums.append(station_num)
                            boarding_stations_id.append(station_id)
                            break
                    if finded == 0:
                        station_num = len(stops_times)
                        station_id = route_stops_arr[len(stops_times)]
                        boarding_stations_nums.append(station_num)
                        boarding_stations_id.append(station_id)
                else:
                    boarding_stations_nums.append(station_num)
                    boarding_stations_id.append(station_id)
            else:
                cur_trip_num = trip_id
                cur_direction = direction_id
                curr_cluster = cluster_id
                
                if cur_direction == 0:
                    stops_times = cur_comm_df['times_for_stops1'].values[0].split(',')
                    route_stops_arr = cur_comm_df['stops_in_direction1'].values[0].split(',')
                else:
                    stops_times = cur_comm_df['times_for_stops2'].values[0].split(',')  
                    route_stops_arr = cur_comm_df['stops_in_direction2'].values[0].split(',')
                    
                station_num = 0
                station_id = route_stops_arr[0]
                boarding_stations_nums.append(station_num)
                boarding_stations_id.append(station_id)
        
                
        stops_num_column = np.array(boarding_stations_nums)
        stops_id_column = np.array(boarding_stations_id)
        
        # Этап 4 - корректировка информации о поездках
        tr_times = board_df['TRANSACT_TIME'].values
        tr_num = tr_times.shape[0]
        
        trip_nums = preliminary_trip_id_column
        directions = preliminary_direction_column
        stops_id = stops_id_column
        stops_numbers = stops_num_column
        clusters = clusters_column
        
        curr_trip = trip_nums[tr_num - 1]
        curr_dir = directions[tr_num - 1]
        curr_number = stops_numbers[tr_num - 1]
        curr_cluster = clusters[tr_num - 1]
        
        new_trips_ids = [curr_trip]
        new_directions = [curr_dir]
        new_clusters = [curr_cluster]
        new_numbers = [curr_number]
        
        pointer = tr_num - 1
        while pointer > 0:
            if (trip_nums[pointer] != trip_nums[pointer - 1]) and (stops_id[pointer] == stops_id[pointer - 1]):
                while stops_id[pointer] == stops_id[pointer - 1]:
                    new_trips_ids.append(curr_trip)
                    new_directions.append(curr_dir)
                    new_clusters.append(curr_cluster)
                    new_numbers.append(curr_number)
                    pointer -= 1
            else:
                curr_trip = trip_nums[pointer - 1]
                curr_dir = directions[pointer - 1]
                curr_cluster = clusters[pointer - 1]
                curr_number = stops_numbers[pointer - 1]
                
                new_trips_ids.append(curr_trip)
                new_directions.append(curr_dir)
                new_clusters.append(curr_cluster)
                new_numbers.append(curr_number)
                pointer -= 1
                
        # Добавление информации, полученной на 1-4 этапах в dataframe
        new_trips_column = pd.Series(np.array(list(reversed(new_trips_ids))), index=board_df.index)
        board_df.loc[:, 'TRIP_ID'] = new_trips_column
        
        new_dirs_column = pd.Series(np.array(list(reversed(new_directions))), index=board_df.index)
        board_df.loc[:, 'DIRECTION'] = new_dirs_column
        
        boarding_clusters = pd.Series(np.array(list(reversed(new_clusters))), index=board_df.index)
        board_df.loc[:, 'BOARDING_CLUSTER'] = boarding_clusters
        
        boarding_stops_nums = pd.Series(np.array(list(reversed(new_numbers))), index=board_df.index) 
        board_df.loc[:, 'BOARDING_STOP_NUM'] = boarding_stops_nums
        
        boarding_stops_ids = pd.Series(stops_id_column, index=board_df.index) 
        board_df.loc[:, 'BOARDING_STOP_ID'] = boarding_stops_ids
        
        # Сохраняем информацию
        board_df.to_csv('resources/' + route_num + '_route_' +  str(board_num) + '_board_itog.csv')

In [ ]:
r = get_boarding_locations_by_day('92', '10.10')

In [ ]:
# Просмотр результата функции get_boarding_locations_by_day
r_df = pd.read_csv('resources/92_route_67_board_itog.csv')
r_df = r_df[['CARD_NUM', 'TRANSACT_DAY', 'TRANSACT_MONTH', 'TRANSACT_YEAR', 'CARRIER_BOARD_NUM', 'TRANSACT_TIME', 'ROUTE_NUM', 'TRIP_ID', 'DIRECTION', 'BOARDING_CLUSTER', 'BOARDING_STOP_NUM', 'BOARDING_STOP_ID']]
r_df

In [ ]:
# Вспомогательная часть кода, генерирующая полный список дат из рассматриваемого периода.
dates_array = []
aug_dates = range(1, 25)
sept_dates = range(1, 31)
oct_dates = range(1, 32)
for date in aug_dates:
    dates_array.append(str(date) + '.8')
for date in sept_dates:
    dates_array.append(str(date) + '.9')
for date in oct_dates:
    dates_array.append(str(date) + '.10')

In [ ]:
# Основная функция алгоритма. Использует вспомогательные функции и для переданного маршрута рассчитывает пассажиропотоки 
# на всех транспортных средствах во все даты, в которые есть информация об этом маршруте
def extract_boarding_locations(route_num):
    for tr_date in dates_array:
        if check_date(route_num, tr_date):
            get_boarding_locations_by_day(route_num, tr_date)

In [ ]:
# Применяем описанный алгоритм по очереди ко всем интересуемым маршрутам
for route_num in choosen_routes:
    extract_boarding_locations(route_num)